In [54]:
from samplers import *

import numpy as np
import sklearn
import sklearn.metrics
import scipy
from torch import nn


In [55]:
class MLP(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"


array([[0.        , 0.76519511],
       [0.        , 0.85707071],
       [0.        , 0.81811303],
       ...,
       [0.        , 0.73195868],
       [0.        , 0.82428942],
       [0.        , 0.56737045]])

In [49]:

def jsd(p, q):
    # find the norm and calculate r
    p = (p / p.sum()).flatten()
    q = (q / q.sum()).flatten()
    r = 0.5 * (p + q)
    
    # find the first kl and do some data wrangling (we get some nans)
    first_kl = p * np.log(p / r)
    first_kl[np.isnan(first_kl)] = 0.
    first_kl = np.sum(first_kl)

    # find the first kl and do some data wrangling (we get some nans)
    second_kl = q * np.log(q / r)
    second_kl[np.isnan(second_kl)] = 0.
    second_kl = np.sum(second_kl)
    
    # find the JSD
    return np.log(2) + first_kl / 2. + second_kl / 2.


In [50]:
jsd(next(distribution3()), next(distribution3()))

/Users/martinweiss/.pyenv/versions/3.5.2/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':
/Users/martinweiss/.pyenv/versions/3.5.2/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  


0.20500252021681908